In [1]:
import os.path as osp
import subprocess
import os
from configs.constants import XMAGICAL_EMBODIMENT_TO_ENV_NAME

from absl import app
from absl import logging
from configs.constants import ALGORITHMS
from configs.constants import EMBODIMENTS
from torchkit.experiment import string_from_kwargs
from torchkit.experiment import unique_id
import yaml
import random


2024-11-27 23:32:44.486389: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-27 23:32:44.486514: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-27 23:32:44.486535: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-27 23:32:44.498188: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
seeds = [0, 5]
device = "cuda:0"

experiment_name = '/home/user/xirl/exp/same_long/'
embodiment = 'longstick'

with open(osp.join(experiment_name, "config.yaml"), "r") as fp:
    kwargs = yaml.load(fp, Loader=yaml.FullLoader)

if kwargs["algorithm"] == "goal_classifier":
    reward_type = "goal_classifier"
else:
    reward_type = "distance_to_goal"

# Map the embodiment to the x-MAGICAL env name.
env_name = XMAGICAL_EMBODIMENT_TO_ENV_NAME[embodiment]

logging.info("Experiment name: %s", experiment_name)

In [ ]:
# Execute each seed in parallel.
procs = []
for seed in range(*list(map(int, seeds))):
    procs.append(
        subprocess.Popen([  # pylint: disable=consider-using-with
            "python",
            "train_policy.py",
            "--experiment_name",
            experiment_name,
            "--env_name",
            f"{env_name}",
            "--config",
            f"configs/xmagical/rl/env_reward.py:{embodiment}",
            "--config.reward_wrapper.pretrained_path",
            f"{experiment_name}",
            "--config.reward_wrapper.type",
            f"{reward_type}",
            "--seed",
            f"{seed}",
            "--device",
            f"{device}",
            '--resume',
        ]))

# Wait for each seed to terminate.
for p in procs:
    p.wait()

2024-11-27 23:33:02.454759: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-27 23:33:02.454838: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-27 23:33:02.454859: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-27 23:33:02.487423: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-27 23:33:02.669519: E te

Loading chipmunk for Linux (64bit) [/home/user/mambaforge/envs/env/lib/python3.11/site-packages/pymunk/libchipmunk.so]
Loading chipmunk for Linux (64bit) [/home/user/mambaforge/envs/env/lib/python3.11/site-packages/pymunk/libchipmunk.so]


/home/user/mambaforge/envs/env/lib/python3.11/site-packages/gym/utils/passive_env_checker.py:174: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.warn(
/home/user/mambaforge/envs/env/lib/python3.11/site-packages/gym/utils/passive_env_checker.py:187: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.warn(
  0%|          | 3/75000 [00:00<47:53, 26.10it/s]I1127 23:33:28.479795 138862981760832 utils.py:114] Restored model from checkpoint 4001.
I1127 23:33:28.482826 138862981760832 utils.py:131] Successfully loaded self_supervised_emb.pkl from /home/user/xirl/exp/same_long/self_supervised_emb.pkl
W1127 23:33:28.488785 131309420304192 checkpoint.py:108] optimizer in saved checkpoint not in checkpoint to reload. Skipping it.
I1127 2

Loading chipmunk for Linux (64bit) [/home/user/mambaforge/envs/env/lib/python3.11/site-packages/pymunk/libchipmunk.so]
Loading chipmunk for Linux (64bit) [/home/user/mambaforge/envs/env/lib/python3.11/site-packages/pymunk/libchipmunk.so]


  0%|          | 2/75000 [00:00<1:11:11, 17.56it/s]/home/user/mambaforge/envs/env/lib/python3.11/site-packages/gym/utils/passive_env_checker.py:219: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(
/home/user/mambaforge/envs/env/lib/python3.11/site-packages/gym/utils/passive_env_checker.py:225: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(done, (bool, np.bool8)):
/home/user/mambaforge/envs/env/lib/python3.11/site-packages/gym/utils/passive_env_checker.py:219: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(
/home/user/mambaforge/envs/env/lib/python3.11/site-packages/gym/utils/passive_env_checker.py:225: DeprecationWarning: `np.bo

Loading chipmunk for Linux (64bit) [/home/user/mambaforge/envs/env/lib/python3.11/site-packages/pymunk/libchipmunk.so]


  0%|          | 26/75000 [00:00<34:55, 35.78it/s]/home/user/mambaforge/envs/env/lib/python3.11/site-packages/gym/utils/passive_env_checker.py:219: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(
/home/user/mambaforge/envs/env/lib/python3.11/site-packages/gym/utils/passive_env_checker.py:225: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(done, (bool, np.bool8)):
 66%|██████▌   | 49607/75000 [1:20:09<30:05, 14.07it/s]    